In [146]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import date

In [265]:
实时行情数据_df = ak.stock_zh_a_spot()

Please wait for a moment: 100%|████████████████████████████████████████████████████████| 54/54 [00:27<00:00,  2.00it/s]


In [148]:
股票代码 = 'sh600157'

In [264]:
股票市价     = 实时行情数据_df[实时行情数据_df['代码']== 股票代码]['最新价']    # 股票市价（元）
#发行在外股份 =      # 发行在外的股份

In [226]:
from datetime import date
today = date.today()
# YYmmdd
today = today.strftime("%Y%m%d")
历史行情数据_df = ak.stock_zh_a_daily(symbol=股票代码,start_date=today,end_date=today,adjust="hfq")
流动股本 = 历史行情数据_df['outstanding_share'][0]    # 流通在外的股份
流动股本

22217764145.0

In [237]:
# 下一年度自由现金流
下一年度自由现金流 = 630000000

In [238]:
永续年金增长率 = 0.03
折现率         = 0.09

In [286]:
# N年估值模型
N = 10

In [287]:
# 第一步：预测下一个N年的自由现金流（FCF）
# 假设增长率保持5%不变
grow_rate=0.05

values = [ round(下一年度自由现金流 * (1+grow_rate)**x) for x in range(N) ]
years  = [ x+1 for x in range(N) ]
# creating a pandas dataframe
df = pd.DataFrame({'年份':years,'自由现金流':values})
df

,年份,自由现金流
0,1,630000000
1,2,661500000
2,3,694575000
3,4,729303750
4,5,765768938
5,6,804057384
6,7,844260254
7,8,886473266
8,9,930796930
9,10,977336776


In [288]:
# 第二步：把这些自由现金流折现成现值
# 折现因子=(1+R)^N，其中R是折现率，N是折现的年数
df['折现因子']=(1+折现率)**df.年份
df['折现自由现金流']=df.自由现金流/df.折现因子
df

,年份,自由现金流,折现因子,折现自由现金流
0,1,630000000,1.090000,5.779817e+08
1,2,661500000,1.188100,5.567713e+08
2,3,694575000,1.295029,5.363393e+08
3,4,729303750,1.411582,5.166572e+08
4,5,765768938,1.538624,4.976973e+08
5,6,804057384,1.677100,4.794331e+08
6,7,844260254,1.828039,4.618393e+08
7,8,886473266,1.992563,4.448910e+08
8,9,930796930,2.171893,4.285648e+08
9,10,977336776,2.367364,4.128376e+08


In [289]:
# 第三步：计算永续年金价值并把它折现成现值
# 永续年金价值 (PV) = YrNFCF * (1+g) / (R-g)，其中g是永续年金增长率，R是折现率
YrNFCF = df[df['年份']==N]['自由现金流']   # 第N年的自由现金流
永续年金价值 = YrNFCF * (1 + 永续年金增长率) / (折现率 - 永续年金增长率)
折现         = 永续年金价值/df[df['年份']==N]['折现因子']
折现

9    7.087046e+09
dtype: float64

In [290]:
# 第四步：计算所有者权益合计
# N年折现现金流（第二步）加上永续年金的折现值（上一步）
所有者权益合计 = df['折现自由现金流'].sum() + 折现
所有者权益合计/1000000

9    12000.05832
dtype: float64

In [291]:
# 第五步：计算每股价值
每股价值 = 所有者权益合计 / 流动股本
每股价值

9    0.540111
dtype: float64